In [2]:

import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, stats
import networkx as nx
from sklearn.decomposition import PCA
import community as community_louvain  # python-louvain
import warnings
import math
import pathlib

In [3]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
DATA_DIR =r"C:\Users\rayom\OneDrive\Documentos\PROYECTO MODELOS I\multitaper-spectra-recorded-during-gabaergic-anesthetic-unconsciousness-1.0.0"
RESULTS_DIR = "results"
BANDS = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta' : (13, 30),
    'gamma': (30, 50)
}
PROPORTIONAL_THRESHOLD = 0.20  # retener top 20% de aristas si se usa umbral proporcional
N_SURROGATES = 100  # para test estadístico; aumentar si tienes tiempo computacional

In [11]:
def load_subject_csv(subject_folder):
    subject_data = {}
    csv_files = glob.glob(os.path.join(subject_folder, "*.csv"))

    for csv_file in csv_files:
        name = os.path.basename(csv_file).replace(".csv", "")

        df = pd.read_csv(csv_file, header=None)
        subject_data[name] = df.values.squeeze()

    subject_data["folder"] = subject_folder
    subject_data["id"] = os.path.basename(subject_folder)

    return subject_data

def load_dataset_csv(data_dir):
    subject_folders = glob.glob(os.path.join(data_dir, "**"), recursive=True)
    subject_folders = [f for f in subject_folders if os.path.isdir(f)]

    dataset = []

    for folder in subject_folders:
        csv_files = glob.glob(os.path.join(folder, "*.csv"))
        if len(csv_files) == 0:
            continue

        dataset.append(load_subject_csv(folder))

    print(f"Total de sujetos cargados: {len(dataset)}")
    return dataset


In [16]:
dataset = load_dataset_csv(DATA_DIR)

Total de sujetos cargados: 2


In [20]:
print([k for k in example.keys() if "_Sdb" in k])


['1014_Sdb', '1029_Sdb', '1051_Sdb', '1069_Sdb', '1071_Sdb', '132_Sdb', '133_Sdb', '143_Sdb', '154_Sdb', '208_Sdb', '212_Sdb', '222_Sdb', '223_Sdb', '228_Sdb', '244_Sdb', '281_Sdb', '300_Sdb', '373_Sdb', '455_Sdb', '511_Sdb', '54_Sdb', '62_Sdb', '630_Sdb', '65_Sdb', '695_Sdb', '697_Sdb', '703_Sdb', '713_Sdb', '724_Sdb', '731_Sdb', '788_Sdb', '790_Sdb', '796_Sdb', '806_Sdb', '808_Sdb', '811_Sdb', '834_Sdb', '839_Sdb', '856_Sdb', '858_Sdb', '867_Sdb', '908_Sdb', '937_Sdb', '962_Sdb']


In [17]:
xample = dataset[0]
print(example.keys())

dict_keys(['1014_EEGquality', '1014_f', '1014_l', '1014_Sdb', '1014_t', '1029_EEGquality', '1029_f', '1029_l', '1029_Sdb', '1029_t', '1051_EEGquality', '1051_f', '1051_l', '1051_Sdb', '1051_t', '1069_EEGquality', '1069_f', '1069_l', '1069_Sdb', '1069_t', '1071_EEGquality', '1071_f', '1071_l', '1071_Sdb', '1071_t', '132_EEGquality', '132_f', '132_l', '132_Sdb', '132_t', '133_EEGquality', '133_f', '133_l', '133_Sdb', '133_t', '143_EEGquality', '143_f', '143_l', '143_Sdb', '143_t', '154_EEGquality', '154_f', '154_l', '154_Sdb', '154_t', '208_EEGquality', '208_f', '208_l', '208_Sdb', '208_t', '212_EEGquality', '212_f', '212_l', '212_Sdb', '212_t', '222_EEGquality', '222_f', '222_l', '222_Sdb', '222_t', '223_EEGquality', '223_f', '223_l', '223_Sdb', '223_t', '228_EEGquality', '228_f', '228_l', '228_Sdb', '228_t', '244_EEGquality', '244_f', '244_l', '244_Sdb', '244_t', '281_EEGquality', '281_f', '281_l', '281_Sdb', '281_t', '300_EEGquality', '300_f', '300_l', '300_Sdb', '300_t', '373_EEGqual

In [21]:
# Encuentra la clave que termina en _Sdb
sdb_key = [k for k in example.keys() if k.endswith("_Sdb")][0]

# Extrae el ID quitando _Sdb
subject_id = sdb_key.replace("_Sdb", "")

print("ID real:", subject_id)

# Ahora sí cargas todo
Sdb = example[f"{subject_id}_Sdb"]
f    = example[f"{subject_id}_f"]
t    = example[f"{subject_id}_t"]
l    = example[f"{subject_id}_l"]

print("Sdb shape:", Sdb.shape)
print("Frecuencias:", f[:10])
print("Tiempo:", t[:10])
print("Label:", l)



ID real: 1014
Sdb shape: (100, 6570)
Frecuencias: [0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5]
Tiempo: [ 0.  2.  4.  6.  8. 10. 12. 14. 16. 18.]
Label: [1. 1. 1. ... 0. 0. 0.]
